# Hands On 1 - Introduction to Pyspark

The objective of this laboratory is to start playing around with Apache Spark.

## 0. Warm up

In this exercise you will run a simple Spark program in a Jupyter notebook.
Create a **SparkContext**, then execute the following lines of code which sum the integers:

In [ ]:
# if 'sc' not exists: create
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master('local[1]').appName('Lab1').getOrCreate()
# sc = spark.sparkContext

sc

In [ ]:
rdd = sc.textFile("data/inputdata1.txt")
print(f"The elements of the RDD are in the form:\n{rdd.take(3)}\n")
fieldsRdd = rdd.map(lambda line: line.split(","))
valueRdd = fieldsRdd.map(lambda l: int(l[1]))
valueSum = valueRdd.reduce(lambda v1, v2: v1+v2)
print("The sum is:", valueSum)

Now, modify the previous code to do the following:

- Filter the values greater than  5
- Compute the product of those lines

In [ ]:
### WRITE YOUR CODE HERE ###
filteredRdd = valueRdd.filter(lambda v: v>5)
filteredProd = filteredRdd.reduce(lambda v1, v2: v1*v2)
print(f"The product is: {filteredProd}")

Find how many distinct characters are used in the names in the input dataset. 

**Remember** that to do this task, you need to take the name from each row of the rdd, and than make it _flat_ to get one rdd of single letters.

In [ ]:
### WRITE YOUR CODE HERE ###
charactersRdd = fieldsRdd.flatMap(lambda l: l[0]).distinct()
print("The number of distinct characters is:", charactersRdd.count())

## 1. Word count

In this first part, you want to create a word count application and apply it to a sample text. 
This means that you have to read the file, split it into words and get how many times each word occurs.

You finally have to store the result in the format `word\tfreq`.

**Remember:** differently from the previous task, you now need to perform a count _for each word_. How to do that?



In [ ]:
import re # regex can be used to remove punctuation ('word' and 'word,' are the same)

danteRdd = sc.textFile("data/commedia.txt")

### WRITE YOUR CODE HERE ###
# words_rdd = dante_rrd.flatMap(lambda l: l.split())
wordsRdd = danteRdd.flatMap(lambda l: re.sub(r"[^\w\s]", "", l).split())

wordsPairRdd = wordsRdd.map(lambda w: (w, 1))
countsPairRdd = wordsPairRdd.reduceByKey(lambda v1, v2: v1 + v2)
countsPairRdd.map(lambda v: f"{v[0]}\t{v[1]}").saveAsTextFile("out1")

You will probably end up with a folder (`./out1`) with more files inside. 
- Why the result is split into multiple files?

You assume to start with one file with the word frequencies in
the Amazon food reviews dataset, in the format `word\tfreq`, where freq is an integer.
Your task is to write a Spark application to filter these results, analyze the filtered data and
compute some statistics on them.

## 2. Statistics on frequences

You now have a large file with the word frequencies, in the format `word\tfreq`, where freq is an integer and \t means the tabulation character. The result would be the same if you start with `counts_pairRdd`, but please start reading a file again to get practice! If you have problems on the first part, you can start with the sample files in `data/out1`.

Read the data and parse the content. This means separate the `word` and `freq` and convert `freq` to an integer.

Next, print the $3$ most frequent words.

In [ ]:
# Set input and output folders
# I also define a variable containing the prefix I am interested in
inputPath  = "out1/"
outputPath = "out2/" 
prefix = "me"

In [ ]:
# Read input data
wordsFrequenciesRDD = sc.textFile(inputPath)

In [ ]:
# Keep only the lines containing words that start with the prefix “me”
### WRITE YOUR CODE HERE ###
selectedLinesRDD = wordsFrequenciesRDD.filter(lambda line: line.startswith(prefix))

In [ ]:
### WRITE YOUR CODE HERE ###

# Print on the standard output the following statistics
# - The number of selected lines

numLines = selectedLinesRDD.count()
print(f"Num. selected lines: {str(numLines)}" ) 

In [ ]:
### WRITE YOUR CODE HERE ###

# Print on the standard output the following statistics
# - The maximum frequency (maxfreq) among the ones of the selected lines (i.e., the maximum value of freq in the lines obtained by applying the filter).

# Select the values of frequency
maxfreqRDD = selectedLinesRDD.map(lambda line: float(line.split("\t")[1]))

# Compute the maximu value
maxfreq = maxfreqRDD.reduce(lambda freq1, freq2: max(freq1, freq2) )

# Print maxfreq on the standard output
print("Maximum frequency: "+ str(maxfreq) ) 

## 2.2 Extra filters

Extend the previous application. Specifically, in the second part of your application, among the lines selected by the first filter, you have to apply another filter to select only the most frequent words. Specifically, your application must select those lines that contain words with a frequency (_freq_) greater than 80% of the maximum frequency (_maxfreq_) computed before.

Hence, implement the following filter:
- Keep only the lines with a frequency freq greater than 0.8*_maxfreq_.

Finally, perform the following operations on the selected lines (the ones selected by applying both filters):
- Count the number of selected lines and print this number on the standard output
- Save the selected words (without frequency) in an output folder (one word per line)


In [ ]:
### WRITE YOUR CODE HERE ###

# Keep only the lines with a frequency freq greater than 0.8*maxfreq.
selectedLinesMaxFreqRDD = selectedLinesRDD.filter(lambda line: float(line.split("\t")[1])>0.8*maxfreq)

In [ ]:
### WRITE YOUR CODE HERE ###

# Count the number of selected lines and print this number on the standard output
numLinesMaxfreq = selectedLinesMaxFreqRDD.count()
print("Num. selected lines with freq > 0.8*maxfreq: "+ str(numLinesMaxfreq) ) 

In [ ]:
# Select only the words (first field)
selectedWordsRDD = selectedLinesMaxFreqRDD.map(lambda line: line.split("\t")[0])

In [ ]:
# Save the selected words (without frequency) in an output folder (one word per line)
selectedWordsRDD.saveAsTextFile(outputPath)